In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flickr2k/Flickr2K/Flickr2K_LR_bicubic/X3/001323x3.png
/kaggle/input/flickr2k/Flickr2K/Flickr2K_LR_bicubic/X3/001882x3.png
/kaggle/input/flickr2k/Flickr2K/Flickr2K_LR_bicubic/X3/001257x3.png
/kaggle/input/flickr2k/Flickr2K/Flickr2K_LR_bicubic/X3/000451x3.png
/kaggle/input/flickr2k/Flickr2K/Flickr2K_LR_bicubic/X3/000013x3.png
/kaggle/input/flickr2k/Flickr2K/Flickr2K_LR_bicubic/X3/001202x3.png
/kaggle/input/flickr2k/Flickr2K/Flickr2K_LR_bicubic/X3/002014x3.png
/kaggle/input/flickr2k/Flickr2K/Flickr2K_LR_bicubic/X3/000282x3.png
/kaggle/input/flickr2k/Flickr2K/Flickr2K_LR_bicubic/X3/001480x3.png
/kaggle/input/flickr2k/Flickr2K/Flickr2K_LR_bicubic/X3/001708x3.png
/kaggle/input/flickr2k/Flickr2K/Flickr2K_LR_bicubic/X3/002374x3.png
/kaggle/input/flickr2k/Flickr2K/Flickr2K_LR_bicubic/X3/001605x3.png
/kaggle/input/flickr2k/Flickr2K/Flickr2K_LR_bicubic/X3/002333x3.png
/kaggle/input/flickr2k/Flickr2K/Flickr2K_LR_bicubic/X3/000912x3.png
/kaggle/input/flickr2k/Flickr2K/Flickr2K_LR_bicu

In [2]:
import os
import glob
import random
import math
from PIL import Image
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
from tqdm import tqdm

##############################
# 1. Dataset Definition
##############################

class SRDataset(Dataset):
    """
    A dataset for single image super-resolution.
    It expects a list of directories containing HR images.
    For each image, a crop is extracted and an LR image is generated via bicubic downsampling.
    """
    def __init__(self, image_dirs, crop_size=96, scale=4, mode='train'):
        self.image_paths = []
        for d in image_dirs:
            self.image_paths.extend(glob.glob(os.path.join(d, "*.png")))
            self.image_paths.extend(glob.glob(os.path.join(d, "*.jpg")))
            self.image_paths.extend(glob.glob(os.path.join(d, "*.jpeg")))
        self.crop_size = crop_size
        self.scale = scale
        self.mode = mode
        
        # Transform: convert PIL Image to Tensor (scaled [0,1])
        self.to_tensor = transforms.ToTensor()
        
        # Data augmentation: For training, apply random crop and horizontal flip (removed rotation)
        if mode == 'train':
            self.augment = transforms.Compose([
                transforms.RandomCrop(crop_size),
                transforms.RandomHorizontalFlip(),
            ])
        else:
            self.augment = transforms.CenterCrop(crop_size)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # Load HR image and convert to RGB
        img_path = self.image_paths[idx]
        hr_image = Image.open(img_path).convert("RGB")
        # Apply augmentation/cropping
        hr_image = self.augment(hr_image)
        # Generate LR image by bicubic downsampling
        lr_size = (hr_image.width // self.scale, hr_image.height // self.scale)
        lr_image = hr_image.resize(lr_size, resample=Image.BICUBIC)
        
        hr_tensor = self.to_tensor(hr_image)
        lr_tensor = self.to_tensor(lr_image)
        
        return lr_tensor, hr_tensor

##############################
# 2. PSNR Calculation Function
##############################

def calculate_psnr(sr, hr, max_val=1.0):
    mse = F.mse_loss(sr, hr)
    if mse == 0:
        return 100
    psnr = 10 * torch.log10((max_val ** 2) / mse)
    return psnr.item()

##############################
# 3. Model Definition: EDSR
##############################

class ResidualBlock(nn.Module):
    """
    A basic residual block for EDSR.
    It consists of two 3x3 convolutional layers with a ReLU in between.
    A scaling factor (typically 0.1) is applied to the residual for stability.
    """
    def __init__(self, n_feats, res_scale=0.1):
        super(ResidualBlock, self).__init__()
        self.res_scale = res_scale
        self.conv1 = nn.Conv2d(n_feats, n_feats, kernel_size=3, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(n_feats, n_feats, kernel_size=3, padding=1)
        
    def forward(self, x):
        res = self.conv1(x)
        res = self.relu(res)
        res = self.conv2(res)
        return x + res * self.res_scale

class EDSR(nn.Module):
    """
    Enhanced Deep Super-Resolution (EDSR) network.
    Architecture:
      - Shallow feature extraction via a conv layer.
      - A trunk of several residual blocks.
      - A conv layer after the residual blocks.
      - A skip connection from the shallow features.
      - An upscaling module using PixelShuffle to upscale the features.
      - A final convolution to generate the output image.
    """
    def __init__(self, scale=4, n_resblocks=32, n_feats=64, res_scale=0.1, in_channels=3):
        super(EDSR, self).__init__()
        self.scale = scale
        # Shallow feature extraction
        self.conv_in = nn.Conv2d(in_channels, n_feats, kernel_size=3, padding=1)
        # Residual blocks
        self.res_blocks = nn.Sequential(*[ResidualBlock(n_feats, res_scale) for _ in range(n_resblocks)])
        # Conv layer after residual blocks
        self.conv_mid = nn.Conv2d(n_feats, n_feats, kernel_size=3, padding=1)
        # Upscaling module: for scale=4, perform two successive upscaling steps (each by factor 2)
        upscaling = []
        if scale in [2, 3]:
            upscaling.append(nn.Conv2d(n_feats, n_feats * (scale ** 2), kernel_size=3, padding=1))
            upscaling.append(nn.PixelShuffle(scale))
        elif scale == 4:
            for _ in range(2):
                upscaling.append(nn.Conv2d(n_feats, n_feats * 4, kernel_size=3, padding=1))
                upscaling.append(nn.PixelShuffle(2))
        else:
            raise NotImplementedError("Scale factor {} not supported.".format(scale))
        self.upscale = nn.Sequential(*upscaling)
        # Final conv layer to produce output image
        self.conv_out = nn.Conv2d(n_feats, in_channels, kernel_size=3, padding=1)
    
    def forward(self, x):
        x = self.conv_in(x)
        residual = x
        x = self.res_blocks(x)
        x = self.conv_mid(x)
        x = x + residual  # global skip connection
        x = self.upscale(x)
        x = self.conv_out(x)
        return x

##############################
# 4. Training Loop
##############################

def train_model(model, train_loader, val_loader, num_epochs, optimizer, criterion, scheduler, device):
    best_val_psnr = 0.0
    for epoch in range(1, num_epochs + 1):
        model.train()
        train_psnr_total = 0.0
        train_batches = 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch} Train")
        for lr, hr in pbar:
            lr, hr = lr.to(device), hr.to(device)
            optimizer.zero_grad()
            sr = model(lr)
            loss = criterion(sr, hr)
            loss.backward()
            # Apply gradient clipping for stability
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            psnr = calculate_psnr(sr, hr)
            train_psnr_total += psnr
            train_batches += 1
            pbar.set_postfix(loss=loss.item(), psnr=psnr)
        
        avg_train_psnr = train_psnr_total / train_batches
        
        # Validation step
        model.eval()
        val_psnr_total = 0.0
        val_batches = 0
        with torch.no_grad():
            pbar_val = tqdm(val_loader, desc=f"Epoch {epoch} Val")
            for lr, hr in pbar_val:
                lr, hr = lr.to(device), hr.to(device)
                sr = model(lr)
                psnr = calculate_psnr(sr, hr)
                val_psnr_total += psnr
                val_batches += 1
                pbar_val.set_postfix(psnr=psnr)
        avg_val_psnr = val_psnr_total / val_batches
        
        # Step the scheduler
        scheduler.step()
        
        print(f"Epoch {epoch}: Train PSNR: {avg_train_psnr:.2f} dB, Val PSNR: {avg_val_psnr:.2f} dB")
        
        # Save checkpoint if validation improves
        if avg_val_psnr > best_val_psnr:
            best_val_psnr = avg_val_psnr
            torch.save(model.state_dict(), "best_edsr_model.pth")
            print(f"--> Saved new best model with Val PSNR: {best_val_psnr:.2f} dB")

##############################
# 5. Main Function
##############################

def main():
    # Directories provided by the user (modify these paths as needed in Kaggle)
    div2k_dir = "/kaggle/input/div2k-dataset/DIV2K_train_HR/DIV2K_train_HR"   # example DIV2K path
    flickr2k_dir = "/kaggle/input/flickr2k/Flickr2K/Flickr2K_HR"                # example Flickr2K path
    image_dirs = [div2k_dir, flickr2k_dir]
    
    # Create dataset
    crop_size = 96  # HR crop size (adjust if desired)
    scale = 4
    full_dataset = SRDataset(image_dirs=image_dirs, crop_size=crop_size, scale=scale, mode='train')
    
    # Split dataset: 90% training, 10% validation
    total_size = len(full_dataset)
    train_size = int(0.9 * total_size)
    val_size = total_size - train_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
    
    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)
    
    # Initialize model, loss function, and optimizer (with weight decay for regularization)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = EDSR(scale=scale, n_resblocks=32, n_feats=64, res_scale=0.1, in_channels=3).to(device)
    criterion = nn.MSELoss()  # L2 loss for PSNR optimization
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
    
    # Set up a learning rate scheduler: decay LR by 0.5 every 50 epochs
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)
    
    num_epochs = 200  # Extended training epochs
    train_model(model, train_loader, val_loader, num_epochs, optimizer, criterion, scheduler, device)

if __name__ == "__main__":
    main()


Epoch 1 Val: 100%|██████████| 22/22 [00:15<00:00,  1.41it/s, psnr=22.3]


Epoch 1: Train PSNR: 17.67 dB, Val PSNR: 21.44 dB
--> Saved new best model with Val PSNR: 21.44 dB


Epoch 2 Val: 100%|██████████| 22/22 [00:12<00:00,  1.71it/s, psnr=22.4]


Epoch 2: Train PSNR: 22.77 dB, Val PSNR: 23.02 dB
--> Saved new best model with Val PSNR: 23.02 dB


Epoch 3 Val: 100%|██████████| 22/22 [00:13<00:00,  1.60it/s, psnr=22.3]


...

Epoch 84: Train PSNR: 25.21 dB, Val PSNR: 25.09 dB


Epoch 85 Val: 100%|██████████| 22/22 [00:12<00:00,  1.73it/s, psnr=24.1]


Epoch 85: Train PSNR: 25.23 dB, Val PSNR: 24.49 dB


Epoch 86 Train:  45%|████▌     | 88/195 [00:50<01:01,  1.74it/s, loss=0.00399, psnr=24]  


KeyboardInterrupt: 